In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.corpus import stopwords
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
emma = nltk.corpus.gutenberg.sents('austen-emma.txt')

emma_sentences = []
emma_word_set = []

for sentence in emma:
    emma_sentences.append([word.lower() for word in sentence if word.isalpha()])
    for word in sentence:
        if word.isalpha():
            emma_word_set.append(word.lower())

emma_word_set = set(emma_word_set)

In [ ]:
def TermFreq(document, word):
    doc_length = len(document)
    occurances = len([w for w in document if w == word])
    return occurances / doc_length

TF(emma_sentences[5], 'ago')

In [ ]:
def build_DF_dict():
    output = {}
    for word in emma_word_set:
        output[word] = 0
        for doc in emma_sentences:
            if word in doc:
                output[word] += 1
    return output
        
df_dict = build_DF_dict()

df_dict['ago']

In [ ]:
def InverseDocumentFrequency(word):
    N = len(emma_sentences)
    try:
        df = df_dict[word] + 1
    except:
        df = 1
    return np.log(N/df)

InverseDocumentFrequency('ago')

In [ ]:
def TFIDF(doc, word):
    tf = TF(doc, word)
    idf = InverseDocumentFrequency(word)
    return tf*idf

print('ago - ' + str(TFIDF(emma_sentences[5],'ago')))
print('indistinct - ' + str(TFIDF(emma_sentences[5],'indistinct')))

In [ ]:
def loadGlove(path):
    file = open(path,'r')
    model = {}
    for l in file:
        line = l.split()
        word = line[0]
        value = np.array([float(val) for val in line[1:]])
        model[word] = value
    return model

glove = loadGlove('glove.6B.50d.txt')

In [ ]:
embeddings = []

for word in emma_sentences[5]:
    embeddings.append(glove[word])

mean_embedding = np.mean(embeddings, axis = 0).reshape(1, -1)

print(mean_embedding)

In [ ]:
embeddings = []

for word in emma_sentences[5]:
    tfidf = TFIDF(emma_sentences[5], word)
    embeddings.append(glove[word]* tfidf) 
    
tfidf_weighted_embedding = np.mean(embeddings, axis = 0).reshape(1, -1)

print(tfidf_weighted_embedding)

In [ ]:
cosine_similarity(mean_embedding, tfidf_weighted_embedding)